In [1]:
import init
import time
import sbf_classes_v1 as sbf
import numpy as np
import sbf_pr as p_r
from astropy.table import Table

t0=time.time()
# galaxies = ['ic0745','ngc4753','ngc5813','ngc5831','ngc5839']
# galaxies = ['vcc1025','vcc0828','vcc1146']
galaxies = ['vcc0122']
# galaxies = ['vcc1146']
# galaxies=['vcc0828']
# bands=['i']
bands=['g','i']
instr=['cfht']

################ Query extinction
from astroquery.ipac.irsa.irsa_dust import IrsaDust
# IrsaDust.clear_cache()
R_V={'g': 3.237,'i': 1.684} #CCM value = a(lambda)+b(lambda)/R_v
# R_V=[1.684]
magzp={'g':30.,'i': 30.}
fwhm={'g':0.,'i': 0.}

alpha={'g':0.,'i': 0.}
#################################
# RUN PART 1 OF PROCEDURE : MODEL FITTING AND SUBTRACTION

bkg_val={'vcc0828': 2.783, 'vcc1146': 0.163}
rms=1 # Weight file is an RMS

In [2]:
for i in range(len(galaxies)) :
    
    for j in range(len(bands)) :
        galaxy=galaxies[i]
        passband=bands[j]
        # print(R_V[bands[j]])
        gxy=galaxy+'_'+passband
        instrname=instr[0]+'_galinit'
        init_instr= getattr(init,instrname)
        init_instr(galaxy,passband)# instrument specific params
        init.default() # default for all galaxies
        init_spec = getattr(init, gxy) #specific initialisation for each galaxy
        init_spec()
        fwhm[bands[j]]=sbf.run_part1(rms=rms, remove_bkg=True) #IS BUILDING MODEL NOW
        #FWHM IS UPDATED CORRECTLY ONLY WHEN PART 1 IS RUN
        print('FWHM AAAAAA', fwhm)
        sbf.run_part2(alpha[bands[j]])

../INPUT/IMAGES/VCC0122/VCC0122_g.fits
Filename: ../INPUT/IMAGES/VCC0122/VCC0122_g.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     107   (1201, 1201)   float32   
Filename: ../INPUT/IMAGES/VCC0122/VCC0122_g_sig.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      71   (1201, 1201)   float32   
Sex_mask is 1
LL corner Mean/Median/STD 3sigma (0.17980603873729706, 0.17229792475700378, 0.5630893707275391)
LR corner Mean/Median/STD 3sigma (0.29640480875968933, 0.2725692689418793, 1.0250405073165894)
UL corner Mean/Median/STD 3sigma (0.2830159068107605, 0.2725791931152344, 0.5614736676216125)
UR corner Mean/Median/STD 3sigma (0.25671952962875366, 0.2269081175327301, 1.0831197500228882)
MEAN/MEDIAN/STD 3sigma of entire frame: (0.5188140869140625, 0.4194835126399994, 0.8596455454826355)
100
LL corner Mean/Median/STD 3sigma (0.17980603873729706, 0.17229792475700378, 0.5630893707275391)
LR c

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


KeyboardInterrupt: 

In [ ]:
for i in range(len(galaxies)) :
    table=IrsaDust.get_query_table(galaxies[i],section='ebv')
        
    E_BV=table['ext SandF mean']   
        
    A=[(R_V[band]*E_BV.value) for band in bands]
    print(A)
        #####
    
        # fwhm=[0.61,0.48] # KEEP ONLY WHEN NOT RUNNING PART 1
        #VCC1025:
    fwhm={'g':0.83,'i': 0.58} # KEEP ONLY WHEN NOT RUNNING PART 1
       
        #VCC0828:
        # fwhm={'g':0.79,'i': 0.57} # KEEP ONLY WHEN NOT RUNNING PART 1
    
        #VCC1146:
        # fwhm={'g':0.89,'i': 0.72} # KEEP ONLY WHEN NOT RUNNING PART 1
    
        # print(fwhm) 
        #### PARAMS FOR APER_CORR
    match_scale=1.2
    threshold=35
    csmin=0.8
        # mbright=19
        # mfaint=22
    match_scale=1.2 #scaling factor to be multiplied to mean FWHM in order to match band catalogs
    
        #### PARAMS FOR EPSF
    cscut= 0.6#0.95 #both bands: for generating PSF
    mbright= 18#16 HSC
    mfaint= 20#21 #20.5 HSC
        # cilow=0.25 NOT USED
        # cihigh=0.75 NOT USED
    
    sbf.run_part3(galaxies[i],bands[0],bands[1],A[0],A[1],
                      magzp[bands[0]],magzp[bands[1]],fwhm[bands[0]],fwhm[bands[1]],
                      match_scale,csmin,mfaint,mbright,threshold)

In [ ]:
for i in range(len(galaxies)):
    sbf.run_part4(galaxies[i],bands[0],bands[1],magzp[bands[0]],magzp[bands[1]],
                    fwhm[bands[0]],fwhm[bands[1]], A[1])
    

In [ ]:
## PSF Modelling:
for i in range(len(galaxies)):
    psf_rad_scale=3 # 5 FOR HSC.#4*plate_scale
    nthfactor=20 # nth neighbor selection radius= nthfactor*rad_asec
    rgc_factor=40

    psfsize=64
    oversampling=4
    sbf.run_part5(galaxies[i],bands[0],bands[1],A[0],A[1],magzp[bands[0]],
                    magzp[bands[1]],fwhm[bands[0]],fwhm[bands[1]],
                    mfaint,mbright,threshold,cscut,psf_rad_scale,nthfactor,
                    psfsize,oversampling,rgc_factor)

In [ ]:
for i in range(len(galaxies)):
# sbf.find_annuli(galaxies[i],bands[0],bands[1])

        
    # MASKING ANNULUS: Inner and outer radii--- in pix
    #VCC1025:
    #ORIGINAL    
    #in_rad=np.array([36,54,90,162,306,36,36])#*0.187#0.00187 # degrees 36 pix   0.0084 #
    #out_rad=np.array([54,90,162,306,594,600,300])#*0.187#0.0156 #0.0159 # cannulus 4 162 to 306 pix # degrees 300 pix : cannulus7 from Mik
    in_rad=np.array([236,254,290,162,306,36,36])#*0.187#0.00187 # degrees 36 pix   0.0084 #
    out_rad=np.array([354,390,362,306,594,600,300])#*0.187#0.0156 #0.0159 # cannulus 4 162 to 306 pix # degrees 300 pix : cannulus7 from Mik

    #VCC0828:
    # in_rad=np.array([85,95,115,155,235,85,85])#*0.187#0.00187 # degrees 36 pix   0.0084 #
    # out_rad=np.array([95,115,155,235,395,385,235])#*0.187#0.0156 #0.0159 # cannulus 4 162 to 306 pix # degrees 300 pix : cannulus7 from Mik
    # in_rad=np.array([85,85])*0.187#0.00187 # degrees 36 pix   0.0084 #
    # out_rad=np.array([385,235])*0.187#0.0156 #0.0159 # cannulus 4 162 to 306 pix # degrees 300 pix : cannulus7 from Mik

    #VCC1146:
    # in_rad=np.array([40,62,106,194,370,40,40])
    # out_rad=np.array([62,106,194,370,722,740,420])

    cutout_size=1536


    sbf.run_pr_cat(galaxies[i],bands[0],bands[1],magzp[bands[0]],magzp[bands[1]],
                    fwhm[bands[0]],fwhm[bands[1]], cutout_size, csmin,mfaint,mbright,threshold)
